# Tutorial 12: Bottleneck Experiments

This tutorial walks you through the process of *running the bottleneck experiments*. The bottleneck experiment, depicted in Fig. 1, is intended to illustrate the dynamics of traffic in a bottleneck. In particular, our bottleneck is intended to imitate the dynamics of traffic on the San Francisco-Oakland Bay Bridge, where fifteen lanes narrow to eight to five. In our bottleneck, we have N * 4 lanes narrow to N * 2 to N, where N is a scaling factor that can be used to increase the number of lanes. As demonstrated in Fig. 2, this bottleneck has a phenomenon known as *capacity drop*: as the number of vehicles flowing into the system increases the number of exiting vehicles initially increases. However, above a critical flow of entering vehicles, the output of the system starts to decrease as congestion forms. Just like in the San Francisco-Oakland Bay Bridge, there is a configurable toll booth and a set of traffic lights that can be used to control the inflow. Each of the merges is implemented as a zipper merge, in which the vehicles merge as late as possible. 

<center>
<img src="img/bottleneck.png">

Figure 1. A visual of the bottleneck that is modeled on the San Francisco-Oakland bay bridge. There are two bottlenecks, the first cuts the number of lanes in half and the second cuts the number of lanes in half again.</center>

<center>
<img src="img/capacity_curve.png">
</center>
<center>
Figure 2. The inflow-outflow relationship for the bottleneck depicted in Fig. 1. As the inflow increases, the outflow steadily ticks up, but above a critical inflow the outflow suddenly ticks down and remains at that value. This phenomenon is known as *capacity drop*. The shaded area indicates 1 std deviation around the outflow.
</center>


The remainder of this tutorial is organized as follows:

* Section 1 introduces the basic configurable parameters of the traffic light environment.
* Section 2 shows how to configure the toll booth.
* Section 3 shows how to use the traffic lights to perform ramp metering.
* Section 4 introduces some autonomous vehicles into the system and describes the Markov Decision Process that the bottleneck comes pre-configured with. 

## 1. Configuring the Environment

Here we describe the different basic configurable parameters of the bottleneck environment. First, we import the necessary classes and parameters to run the environment. We will highlight the effects of changing certain parameters.

In [ ]:
# Import all of the necessary pieces of Flow to run the experiments
from flow.core.params import SumoParams, EnvParams, NetParams, InitialConfig, \
    InFlows, SumoLaneChangeParams, SumoCarFollowingParams
from flow.core.params import VehicleParams
from flow.core.params import TrafficLightParams

from flow.networks.bottleneck import BottleneckNetwork
from flow.controllers import SimLaneChangeController, ContinuousRouter
from flow.envs.bottleneck import BottleneckEnv
from flow.core.experiment import Experiment

import logging

def run_exp(flow_rate,
            scaling=1,
            disable_tb=True,
            disable_ramp_meter=True,
            n_crit=1000,
            feedback_coef=20):
    # Set up SUMO to render the results, take a time_step of 0.5 seconds per simulation step
    sim_params = SumoParams(
        sim_step=0.5,
        render=True,
        overtake_right=False,
        restart_instance=False)

    vehicles = VehicleParams()

    # Add a few vehicles to initialize the simulation. The vehicles have all lane changing enabled, 
    # which is mode 1621
    vehicles.add(
        veh_id="human",
        lane_change_controller=(SimLaneChangeController, {}),
        routing_controller=(ContinuousRouter, {}),
        car_following_params=SumoCarFollowingParams(
            speed_mode=25,
        ),
        lane_change_params=SumoLaneChangeParams(
            lane_change_mode=1621,
        ),
        num_vehicles=1)

    # These are additional params that configure the bottleneck experiment. They are explained in more
    # detail below.
    additional_env_params = {
        "target_velocity": 40,
        "max_accel": 1,
        "max_decel": 1,
        "lane_change_duration": 5,
        "add_rl_if_exit": False,
        "disable_tb": disable_tb,
        "disable_ramp_metering": disable_ramp_meter,
        "n_crit": n_crit,
        "feedback_coeff": feedback_coef,
    }
    # Set up the experiment to run for 1000 time steps i.e. 500 seconds (1000 * 0.5)
    env_params = EnvParams(
        horizon=1000, additional_params=additional_env_params)

    # Add vehicle inflows at the front of the bottleneck. They enter with a flow_rate number of vehicles 
    # per hours and with a speed of 10 m/s
    inflow = InFlows()
    inflow.add(
        veh_type="human",
        edge="1",
        vehsPerHour=flow_rate,
        departLane="random",
        departSpeed=10)

    # Initialize the traffic lights. The meanings of disable_tb and disable_ramp_meter are discussed later.
    traffic_lights = TrafficLightParams()
    if not disable_tb:
        traffic_lights.add(node_id="2")
    if not disable_ramp_meter:
        traffic_lights.add(node_id="3")

    additional_net_params = {"scaling": scaling, "speed_limit": 23}
    net_params = NetParams(
        inflows=inflow,
        additional_params=additional_net_params)

    initial_config = InitialConfig(
        spacing="random",
        min_gap=5,
        lanes_distribution=float("inf"),
        edges_distribution=["2", "3", "4", "5"])

    flow_params = dict(
        exp_tag='bay_bridge_toll',
        env_name=BottleneckEnv,
        network=BottleneckNetwork,
        simulator='traci',
        sim=sim_params,
        env=env_params,
        net=net_params,
        veh=vehicles,
        initial=initial_config,
        tls=traffic_lights,
    )

    # number of time steps
    flow_params['env'].horizon = 1000
    exp = Experiment(flow_params)

    # run the sumo simulation
    _ = exp.run(1)

### The effects of scaling
Setting scaling to values greater than 1 leads to the number of lanes increasing. Scaling=1 means we start with four lanes, scaling=2 leads to us starting wth 8 lanes, etc.

Fig. 3 depicts the effect of scaling on the bottleneck.

<center>
<img src="img/bottleneck_scaling_1.png">
<img src="img/bottleneck_scaling_2.png">

Figure 3. The effects of scaling on the bottleneck. If we set scaling to 1, the number of lanes goes 4 to 2 to 1. If we set scaling to 2, the number of lanes goes 8 to 4 to 2. This pattern continues as we increase the scaling values.
</center>

**scaling=1**

In [ ]:
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=True)

**scaling=2**

In [ ]:
run_exp(flow_rate=1000, scaling=2, disable_tb=True, disable_ramp_meter=True)

### The effects of inflow
Increasing the inflow rate into the system controls whether congestion sets in or not. For values approximately less than 1000, congestion rarely sets in. As the value of the inflow increases, congestion becomes more likely. Around an inflow of around 1600 congestion occurs with high certainty.

In [ ]:
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=True)

In [ ]:
run_exp(flow_rate=1400, scaling=1, disable_tb=True, disable_ramp_meter=True)

In [ ]:
run_exp(flow_rate=2400, scaling=1, disable_tb=True, disable_ramp_meter=True)

## Adding a tollbooth
On the segment indicated in Fig. 4, we have set up a series of rules that allow users to mimic the effect of a tollbooth. If *disable_tb=False*, cars that approach the toll-booth have their color changed to blue to indicate that they are in the toll-booth region. Their lane changing is disabled. As they come to the front of the toll-booth, we sample from a gaussian to determine how long they should be held at the toll booth. The holding process is imitated by a red light that remains red for the duration of the holding time. As indicated in the figure, the outer lanes are fast-trak lanes; their mean holding time is set to be lower than the holding time of the other toll-booth lanes. For the exact values of the holding parameters, please refer to the *BottleneckEnv* class in *flow/envs/bottleneck.py*

<img src="img/bottleneck_guide.png">

<center>Figure 4: A bottleneck with both the toll and the ramp meter enabled</center>

In [ ]:
run_exp(flow_rate=1000, scaling=1, disable_tb=False, disable_ramp_meter=True)

## Adding a ramp meter
As indicated in Fig. 5, we also have a feedback ramp meter that can be used to apply control to the bottleneck. The ramp meter runs the ALINEA algorithm for metering control; in short, the algorithm controls how long the lights are red for as a function of the congestion. If the number of vehicles in section 4, shown in Fig. 6 exceed a critical value then the red time of the light begins to increase so as to restrict the inflow and allow congestion to dissipate. If it decreases below a critical value then the red time of the light begins to decrease. Note that vehicles approaching the light are colored blue and their lane changing is disabled until they pass through the light.

<img src="img/segment_4_highlight.png">

<center>Figure 4: The section whose number of vehicles we use to decide how long the red light is on for.</center>

We highlight a few of the parameters that can be used to modify the ALINEA algorithm:

### n_crit:
*n_crit* sets the number of vehicles around which we try to stabilize the system. If the number of vehicles in section 4 increases above n_crit the red time will begin to increase. If *n_crit* is set too low than the flow will not be maximized as the traffic lights will be too aggressive in restricting inflow; conversely if n_crit is set too high then the traffic lights will not be aggressive enough and the congestion will never dissipate.

In [ ]:
## An example of a good value of n_crit
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=8)

In [ ]:
## An example where n_crit is way too low
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=2)

In [ ]:
## An example where n_crit is way too high
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=30)

### feedback_coef:
*feedback_coef* sets the aggressiveness with which the red light time changes in response to the number of vehicles in section 4 not being at *n_crit*. High values cause the total red light time to shift very aggressively, low values cause it to react more slowly. You can play with the values below and see the effects

In [ ]:
## An example of a relatively stable feedback value
run_exp(flow_rate=1000, scaling=1, disable_tb=False, disable_ramp_meter=False, n_crit=8, feedback_coef=20)

In [ ]:
## An example of a feedback value that will cause the red time to swing wildly
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=8, feedback_coef=200)

In [ ]:
## An example of a feedback value that will cause the red time to change too slowly
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=8, feedback_coef=1)

For a reference on all of the different parameters that can be used to modify the ALINEA algorithm, please refer to the documentation in *BottleneckEnv* in *flow/envs/bottleneck.py*

## Instantiating a reinforcement learning experiment

While there are many different ways you might use autonomous vehicles (AVs) to try to prevent the effect of a capacity drop, here we demonstrate the particular control scheme used in "Lagrangian control through deep-rl: Applications to bottleneck decongestion" by E. Vinitsky, K. Parvate et. al. 

The code referenced is in *examples/exp_configs/rl/singleagent/singleagent_bottleneck.py*.

We insert a flow of autonomous vehicles as a fraction of the total flow of the system. Due to randomness in the inflow, the number of AVs in the system varies.
In this scheme, all of the AVs are controlled by a single controller. However, because neural network controllers necessarily take in a fixed sized input and have a fixed size output, we come up with a state parametrization and action parametrization that can handle the varying number of vehicles.

To create a fixed size set of states, the entire bottleneck is divided into segments, depicted in white in Fig. 7, and in each of those segments we return aggregate statistics about the speed, number, and density of both the vehicles and AVs in each of the lanes in the segment. For the actions, in each of the action-segments, depicted in ref in Fig. 7 we allow the controller to increase or decrease the speed limit of all the AVs in each lane in that segment.  

<img src="img/bottleneck_control_scheme.png">

<center>Figure 7: A depiction of the scheme used to create a fixed size state and action space for the bottleneck. Segments in white correspond to the division of the bottleneck for states, and segments in red for the division into action. The key at the bottom left indicates the actual information used in the state space for each lane. The numbers for each lane indicate the values that the information would take on for that lane.</center>

### Code
Note, for the following code to run it is necessary to have RLlib installed. It should be installed if you have completed the [Flow setup instructions](https://flow.readthedocs.io/en/latest/flow_setup.html#local-installation-of-flow).

The relevant parameters are:
- symmetric: If true, the same action is applied to every lane in each segment
- controlled_segments: A list of tuples describing wheher a segment is controlled, how many divisions each segment is broken up into, and the name of the segment. For example, controlled_segments = `[("1", 1, False), ("2", 2, True), ("3", 2, True), ("4", 2, True), ("5", 1, False)]` indicates that segment 1 is uncontrolled, segment 2 is controlled and is divided into two pieces, etc.
- observed segments: : A list of tuples describing wheher a segment is observed, how many divisions each segment is broken up into, and the name of the segment. For example, observed_segments = `[("1", 1, False), ("2", 2, True), ("3", 2, True), ("4", 2, True), ("5", 1, False)]` indicates that segment 1 is uncontrolled, segment 2 is controlled and is divided into two pieces, etc.
- reset_inflow: If true, the inflow is randomized for every rollout.
- inflow_range: This contains the range of inflows from which inflows are uniformly sampled if reset_inflow is true.

## Multi-agent reinforcement learning experiment
### In progress